In [1]:
import dlib
import cv2
import numpy as np
from imutils import face_utils

In [2]:
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("C:/Users/mayank singh/Desktop/Opencv/shape_predictor_68_face_landmarks.dat")

In [3]:
def pupil(Eye):
    cv2.drawContours(mask, Eye, -1, (255, 255, 225), -1)
    result = np.bitwise_and(frame,mask)
    result[np.where((result==[0,0,0]).all(axis=2))] = [255,255,255]
    ret,thresh1=cv2.threshold(cv2.cvtColor(result,cv2.COLOR_BGR2GRAY),25,255,cv2.THRESH_BINARY)
    _,contours,_=cv2.findContours(thresh1,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    n=len(contours)-1
    return np.mean(np.array([[cv2.boundingRect(c)[0], cv2.boundingRect(c)[1],cv2.boundingRect(c)[2]+cv2.boundingRect(c)[0],cv2.boundingRect(c)[3]+ cv2.boundingRect(c)[1]] for c in sorted(contours,key=cv2.contourArea,reverse=False)[:len(contours)-1] 
]),axis = 0)

In [7]:
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

cap=cv2.VideoCapture(0)

while True:
    ret, frame=cap.read()
    frame =cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    mask = np.zeros_like(frame)
      
    for subject in subjects:
  
        shape = face_utils.shape_to_np(predict(gray, subject))
        leftEye = shape[rStart:rEnd]  
        pointl=pupil([leftEye])
        
        if pointl.size==4 :     

            if [int((pointl[0]+pointl[2])/2),int((pointl[1]+pointl[3])/2)][0]  in range(leftEye[0][0],leftEye[1][0]) or [int((pointl[0]+pointl[2])/2),int((pointl[1]+pointl[3])/2)][0] in range(leftEye[0][0],leftEye[5][0]):
                cv2.putText(frame, "Left", (25, 75),cv2.FONT_HERSHEY_SIMPLEX, 2 , (0, 150, 0), 5)
            elif [int((pointl[0]+pointl[2])/2),int((pointl[1]+pointl[3])/2)][0] in range(leftEye[4][0],leftEye[3][0]) or [int((pointl[0]+pointl[2])/2),int((pointl[1]+pointl[3])/2)][0] in range(leftEye[2][0],leftEye[3][0]) :
                cv2.putText(frame, "right", (25, 75),cv2.FONT_HERSHEY_SIMPLEX, 2 , (225, 0, 0), 5) 
            elif [int((pointl[0]+pointl[2])/2),int((pointl[1]+pointl[3])/2)][0] in range(leftEye[5][0]+1,leftEye[4][0]-2) and [int((pointl[0]+pointl[2])/2),int((pointl[1]+pointl[3])/2)][0] in range(leftEye[1][0]+1,leftEye[2][0]-2) :
                cv2.putText(frame, "straight", (25, 75),cv2.FONT_HERSHEY_SIMPLEX, 2 , (0, 0, 225), 5)                
           
        

    cv2.imshow("Frame",frame)
    if cv2.waitKey(1)==13:
        break
        
    
cap.release()
cv2.destroyAllWindows()
